# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center> Multi-procesos y Multi-hilos en Python<center></span>

<img src="../images/dask_horizontal.svg" align="right" width="30%">


##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

4. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Contenido</span>


* [Introducción](#Introducción)
* [Multiprocesos](#Multiprocesos)
* [Multihilos](#Multihilos)
* [Creando una sección crítica y Lock para compartir datos](#Creando-una-sección-crítica-y-Lock-para-compartir-datos)
* [Usando un grupo (pool) de trabajadores](#Usando-un-grupo-(pool)-de-trabajadores)

## <span style="color:blue">Fuente</span>

Esta es una adaptación libre del tutorial disponible en [Multiprocesos en Python](https://www.youtube.com/watch?v=ptiAjwyqm_s).

## <span style="color:blue">Introducción</span>

En la primera parte de esta leccion estudiamos comos construir y lanzar múltiples proceso en paralelo con Python.

Los múltiple procesos corren independientemente en cada `core`disponible en su máquina y usan memoria propia, es decir no comparten memoria entre sí. Además estos procesos pueden ser detenidos individualmente.

La ejecución de los multiprocesos se basan en el [GIL](https://blockgeni.com/tutorial-python-global-interpreter-lock/) (Global Interpreter Lock). Un bloqueo de intérprete global (GIL) es un mecanismo para aplicar un bloqueo global a un intérprete. Se utiliza en intérpretes de lenguaje de computadora para sincronizar y administrar la ejecución de subprocesos de modo que solo se pueda ejecutar un subproceso nativo (programado por el sistema operativo) a la vez.

En un escenario en el que tiene varios subprocesos, lo que puede suceder es que ambos subprocesos intenten adquirir la memoria al mismo tiempo y, como resultado, sobrescriban los datos en la memoria. De ahí surge la necesidad de contar con un mecanismo que pueda ayudar a prevenir este fenómeno.

En programación de multiprocesos se usa un GIL en cada proceso. Por los requerimientos, los multiprocesos se demoran más al iniciar que los múltiple hilos (`multi-threads`) 

Por su parte los hilos correspnden a múltiples caminos que puede seguir un proceso y que pueden ejecutarse en paralelo. Lo hilos dependen de un proceso y no pueden interumpirse. A diferencia de los procesos, los hilos si pueden compartir memoria.


## <span style="color:blue">Multiprocesos</span>

### Importa módulos

In [2]:
from multiprocessing import Process, Value
import os

### Crea una función para correr en todos los módulos en paralelo

In [ ]:
Reemplace esta función por una función útil en su caso.

In [3]:
def function(numero):
    print(os.getpid())
    valor = 0
    for n in range(10):
        valor += n * n + n
        print(valor, '-->',numero)

In [225]:
function(2)

8460
0 --> 2
2 --> 2
8 --> 2
20 --> 2
40 --> 2
70 --> 2
112 --> 2
168 --> 2
240 --> 2
330 --> 2


### Detecta los núcleos de la máquina

Ya estamos listos. Empezamos creando una lista de procesos e investigamos el número de núcleos de la máquina.

In [4]:
procesos = []
cores = os.cpu_count()
print('Esta máquina tiene ', cores, 'núcleos')

Esta máquina tiene  4 núcleos


### Instanciar los procesos

In [5]:
for n in range(cores):
    proceso = Process(target=function, args=(n,))
    procesos.append(proceso)

print('... Ejecutar ...')

... Ejecutar ...


In [6]:
procesos

[<Process name='Process-1' parent=3894 initial>,
 <Process name='Process-2' parent=3894 initial>,
 <Process name='Process-3' parent=3894 initial>,
 <Process name='Process-4' parent=3894 initial>]

### Iniciar los procesos

In [7]:
print('Espera hasta terminar...')
for proceso in procesos:
    proceso.start()


Espera hasta terminar...
1044210443

0 --> 10446
00
104512 -->0
  0 --> 1-->
2-->  --> 3 
 10

8822 
  2-->--> --> 1  
0--> 203
2
 
88 20 -->-->  --> 0--> 1 2
20 --> 3


2
40 4040 20  -->-->-->-->  0 
1270
 
70 703
  -->-->  -->040
 --> 2112
1 
 --> 1123112  0-->
70-->  -->  
1168
32
168168  
-->  --> 2112 
-->--> 30
240 1681 

-->240   3-->-->
  2402401 
2 -->
 -->0330
  330330  3-->--> 0

-->  1
330 2
--> 3


### Regreso a la ejecución inicial

Termina los procesos

In [8]:
for proceso in procesos:
    proceso.join()
print('Regreso a la ejecución incial')

Regreso a la ejecución incial


### Retornando valores de un proceso paralelo

Primero crea un objeto de tipo Value en donde el proceso regresará el valor. Este objeto es pasado a la función de trabajo. Allí se deposita el valor a retornar en la propiedad *value*:

In [15]:
ret_value = Value("d", 0.0, lock=False)

def funcion(ret_value, my_value):
    ret_value.value = my_value*2

reader_process = Process(target=funcion, args=[ret_value, 200.])


In [13]:
reader_process.start()
reader_process.join()

In [14]:
ret_value.value

400.0

### Ahora paralelicemos completamente el proceso

In [238]:
# lista de procesos
procesos = []
output = []
cores = os.cpu_count()
print('Esta máquina tiene ', cores, 'núcleos')


# instancia objetos ctype con Value para regresar los valores
for n in range(cores):
    output.append(Value("d", 0.0, lock=False))

# instancia los procesos
j= 100
for n in range(cores):
    proceso = Process(target=funcion, args=(output[n], j))
    procesos.append(proceso)
    j += 10

print(procesos)
print(output)

Esta máquina tiene  4 núcleos
[<Process name='Process-117' parent=8460 initial>, <Process name='Process-118' parent=8460 initial>, <Process name='Process-119' parent=8460 initial>, <Process name='Process-120' parent=8460 initial>]
[c_double(0.0), c_double(0.0), c_double(0.0), c_double(0.0)]


In [239]:
print('lanza los procesos...')
for proceso in procesos:
    proceso.start()

# regresa a la ejecución inicial
for proceso in procesos:
    proceso.join()

print('Procesos terminados')

lanza los procesos...
Procesos terminados


### Muestra el resultado de todos los procesos

In [240]:
for value in output:
    print(value.value)

200.0
220.0
240.0
260.0


## <span style="color:blue">Multihilos</span>

In [171]:
from threading import Thread
import os
import time

In [172]:
# lista de hilos

hilos = []

### Instanciar los hilos

In [173]:
for n in range(cores):
    hilo = Thread(target=function, args=(n,))
    hilos.append(hilo)

print('... Ejecutar ...')

... Ejecutar ...


In [174]:
hilos

[<Thread(Thread-4, initial)>,
 <Thread(Thread-5, initial)>,
 <Thread(Thread-6, initial)>,
 <Thread(Thread-7, initial)>]

### Iniciar y finalizar los hilos

In [175]:
print('Espera hasta terminar...')
for hilo in hilos:
    hilo.start()

for hilo in hilos:
    hilo.join()

print('Regreso a la ejecución inicial')    



Espera hasta terminar...
8460
0 --> 0
2 --> 0
8460
0 --> 1
2 --> 1
8 --> 1
20 --> 1
40 --> 1
70 --> 1
112 --> 1
168 --> 1
240 --> 1
330 --> 1
8 --> 0
20 --> 0
40 --> 0
70 --> 0
112 --> 0
168 --> 0
240 --> 0
330 --> 0
8460
0 --> 2
2 --> 2
8 --> 2
20 --> 2
40 --> 2
70 --> 2
112 --> 2
168 --> 2
240 --> 2
330 --> 2
8460
0 --> 3
2 --> 3
8 --> 3
20 --> 3
40 --> 3
70 --> 3
112 --> 3
168 --> 3
240 --> 3
330 --> 3
Regreso a la ejecución inicial


### Un ejemplo de hilos usando colas para recibir la salida de los hilos

In [16]:
from queue import Queue
from threading import Thread

def foo(bar):
    print('hello {0}'.format(bar))
    return 'foo'

que = Queue()

t = Thread(target=lambda q, arg1: q.put(foo(arg1)), args=(que, 'world!'))
t.start()
t.join()
result = que.get()
print(result)

hello world!
foo


### Compartir datos en hilos

En esta sección vemos lo que ocurre cuando compartimos una variable global (memoria) entre varios hilos

In [185]:
from threading import Thread
import time

In [188]:
# variable global a compartir
valor = 0

In [191]:
# definimos el método (función) que usaran los hilos
def procesa():
    global valor
    # copia local de la variable global
    varLocal =  valor
    # proceso con la variable local
    varLocal += 1
    # simula otro proceso adicional
    time.sleep(0.1)
    # después del proceso, actualizamos la variable global compartida
    valor = varLocal

In [192]:
# valor global inicial
print('Valor al inicio ', valor)

Valor al inicio  0


In [193]:
# define dos hilos
hilo1 = Thread(target=procesa)
hilo2 = Thread(target=procesa)

# lanza los hilos
hilo1.start()
hilo2.start()

# temina los hilos
hilo1.join()
hilo2.join()

In [194]:
# valor global  final
print('Valor al final ', valor)

Valor al final  1


### ¿Qué ocurrió?

Esperabamos un valor final de 2, pero el valor final fue 1. Recuerde que los procesos manejan sus propios recursos de manera separada a los demás procesos y que cada proceso tiene uno o más hilos. Por lo menos tiene un hilo principal. La imagen ilustra un proceso con varios hilos. El problema en nuestro ejercicio anterior es que el sistema operativo (S.O) otorga en secuencia fragmentos de tiempo a cada hilo. El usuario tiene la sensación de que el proceso es paralelo, pero en realidad no lo es. Es esta caso, al colocar el proceso dado por *time.sleep*, toma mucho tiempo y el S.O entrega el control al otro hilo.

![Hilos](../Imagenes/threads_400.png)

Vamos a modificar la función procesa, imprimiendo el valor dentro de cada hilo de cada  y observemos lo que sucede.

In [201]:
# definimos el método (función) que usaran los hilos
def procesa():
    global valor
    # copia local de la variable global
    varLocal =  valor
    print(' valor global leído en el hilo ', varLocal)
    # proceso con la variable local
    varLocal += 1
    # simula otro proceso adicional
    time.sleep(0.1)
    # después del proceso, actualizamos la variable global compartida
    valor = varLocal

In [205]:
# define dos hilos
valor =0
print('Valor al inicio ', valor)

hilo1 = Thread(target=procesa)
hilo2 = Thread(target=procesa)

# lanza los hilos
hilo1.start()
hilo2.start()

# temina los hilos
hilo1.join()
hilo2.join()

print('Valor al final ', valor)

Valor al inicio  0
 valor global leído en el hilo  0 valor global leído en el hilo  0

Valor al final  1


In [ ]:
Ahora vamos a quitar el segundo proceso en lña función y observamos

In [204]:
# definimos el método (función) que usaran los hilos
def procesa():
    global valor
    # copia local de la variable global
    varLocal =  valor
    print(' valor global leído en el hilo ', varLocal)
    # proceso con la variable local
    varLocal += 1
    # simula otro proceso adicional
    #time.sleep(0.1)
    # después del proceso, actualizamos la variable global compartida
    valor = varLocal

In [206]:
# define dos hilos
valor =0
print('Valor al inicio ', valor)

hilo1 = Thread(target=procesa)
hilo2 = Thread(target=procesa)

# lanza los hilos
hilo1.start()
hilo2.start()

# temina los hilos
hilo1.join()
hilo2.join()

print('Valor al final ', valor)

Valor al inicio  0
 valor global leído en el hilo  0
 valor global leído en el hilo  1
Valor al final  2


Lo que comprueba lo dicho arriba. Queda entonces la tarea de ver como evitar el problema presentado aquí.

## <span style="color:blue">Creando una sección crítica y Lock para compartir datos</span>

### Compartir datos en hilos

Estudiamos como crear una sección crítica y el uso de Lock en un hilo. Una sección crítica es una sección de código escrito de tal manera que solamente un hilo pueda ejecutar esa sección a la vez. con *Lock* podemos crear la sección crítica.

Junto con Lock debemos indicar en donde comienza y en donde termina la sección crítica.

In [212]:
from threading import Thread, Lock
import time

In [209]:
# variable global que vamos a compartir
valor = 0

In [210]:
# definimos el método (función) que usaran los hilos
def procesa(lock): # lock será un objeto de tipo Lock
    # valor global a compartir
    global valor
    
    # inicio de la sección crítica
    lock.acquire()
    
    # copia local de la variable global
    varLocal =  valor
    

    #print(' valor global leído en el hilo ', varLocal)
    # proceso con la variable local
    varLocal += 1
    # simula otro proceso adicional
    time.sleep(0.1)
    # después del proceso, actualizamos la variable global compartida
    valor = varLocal
    
    # final de l sección crítica
    lock.release()
    

####  Ejecución con Lock para definir una sección crítica

In [213]:
print('valor de inicio ', valor)

# crea el objeto Lock
lock = Lock()

# define dos hilos
hilo1 = Thread(target=procesa, args=(lock,))
hilo2 = Thread(target=procesa, args=(lock,))

# lanza los hilos
hilo1.start()
hilo2.start()

# temina los hilos
hilo1.join()
hilo2.join()

print('Valor al final ', valor)

valor de inicio  0
Valor al final  2


### Compartir datos en procesos

In [214]:
from multiprocessing import Process, Value
import time

In [219]:
# Definimos el proceso que correremos en paralelo

def proceso(pNumero):
    for n in range(100):
        time.sleep(0.01)
        pNumero.value += 1
        

#### Ejecución principal

In [220]:
# Creamos el objeto de tipo ctype con Value
numero = Value('i',0)

# Creamos instancias de los procesos

p1 = Process(target=proceso, args=(numero,))
p2 = Process(target=proceso, args=(numero,))

# lanza y termina los procesos
p1.start()
p2.start()

p1.join()
p2.join()

print('Numero al final', numero.value)

Numero al final 172


#### Creación de la sección critica.

Como vemos el resultado no es el esperado como sucedióncon los hilos. Será necesario entonces pasar un objeto Lock a la función *proceso* y definir una sección crítica adentro. Veámos:

In [221]:
# Definimos el proceso que correremos en paralelo, incluyeno un objeto Lock como parámetro

def proceso(pNumero, lock):
    for n in range(100):
        time.sleep(0.01)
        # sección crítica
        lock.acquire()
        pNumero.value += 1
        lock.release()
   

#### Ejecución principal

In [223]:
# Creamos el objeto Value
numero = Value('i',0)
print('Numero al comienzo', numero.value)


# creamos el objeto Lock
lock = Lock()

# Creamos instancias de los procesos

p1 = Process(target=proceso, args=(numero, lock))
p2 = Process(target=proceso, args=(numero, lock))

# lanza y termina los procesos
p1.start()
p2.start()

p1.join()
p2.join()

print('Numero al final', numero.value)

Numero al comienzo 0
Numero al final 200


## <span style="color:blue">Usando un grupo (pool) de trabajadores </span>

El objeto `Pool`, que ofrece un medio conveniente para paralelizar la ejecución de una función a través de múltiples valores de entrada, distribuyendo los datos de entrada entre procesos (paralelismo de datos).

La clase Pool representa un grupo de procesos de trabajo. Tiene métodos que permiten que las tareas se descarguen a los procesos de trabajo de diferentes formas.

In [1]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 secs
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
0
1
9
4
16
25
36
49
64
81
400
33857
[33858, 33859, 33860, 33857]
We lacked patience and got a multiprocessing.TimeoutError
For the moment, the pool remains available for more work
Now the pool is closed and no longer available


Eso es todo!!!